# Customer Churn Feature Analysis

This notebook analyzes churn patterns with focus on tenure, usage, and complaint-related features.

## 1. Imports

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')

## 2. Load Data

In [ ]:
base = Path.cwd()
candidates = [
    base / 'clean_telco_churn.csv',
    base / 'notebooks' / 'clean_telco_churn.csv',
    base / 'Customer-Churn-Analysis' / 'notebooks' / 'clean_telco_churn.csv',
    base.parent / 'notebooks' / 'clean_telco_churn.csv',
    base.parent / 'Customer-Churn-Analysis' / 'notebooks' / 'clean_telco_churn.csv',
]

data_path = next((p for p in candidates if p.exists()), None)
if data_path is None:
    raise FileNotFoundError('Could not find clean_telco_churn.csv')

df = pd.read_csv(data_path)
print('Loaded:', data_path)
print('Shape:', df.shape)
df.head()


## 3. Data Quality and Cleaning

In [ ]:
# Standardize column names for easier coding
df.columns = [c.strip().replace(' ', '_') for c in df.columns]

# Basic cleanup for key categorical columns
cat_cols = [
    'Churn_Label', 'Contract', 'Payment_Method', 'Internet_Type',
    'Offer', 'Churn_Category', 'Churn_Reason', 'Customer_Status'
]
for col in cat_cols:
    if col in df.columns:
        # Preserve missing values and avoid string 'nan' artifacts
        s = df[col].astype('string').str.strip()
        df[col] = s.fillna('Unknown')

# Numeric conversion for analysis features
num_cols = [
    'Tenure_in_Months', 'Monthly_Charge', 'Avg_Monthly_GB_Download',
    'Total_Revenue', 'Satisfaction_Score', 'Churn_Score'
]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop exact duplicates
before = len(df)
df = df.drop_duplicates().copy()
after = len(df)

# Create target flag
df['is_churn'] = (df['Churn_Label'].str.lower() == 'yes').astype(int)

print(f'Duplicates removed: {before - after}')
print('Missing values (top 10):')
print(df.isna().sum().sort_values(ascending=False).head(10))


## 4. Core KPIs

In [ ]:
total_customers = len(df)
churned_customers = int(df['is_churn'].sum())
churn_rate = 100 * churned_customers / total_customers

print('Total customers:', total_customers)
print('Churned customers:', churned_customers)
print(f'Churn rate: {churn_rate:.2f}%')

## 5. Feature Comparison

In [ ]:
# Tenure comparison
tenure_comp = df.groupby('Churn_Label')['Tenure_in_Months'].mean().round(2)
print('Average tenure by churn status:')
print(tenure_comp)

# Usage/revenue comparison
usage_cols = ['Monthly_Charge', 'Avg_Monthly_GB_Download', 'Total_Revenue', 'Satisfaction_Score', 'Churn_Score']
usage_comp = df.groupby('Churn_Label')[usage_cols].mean().round(2)
print('\nUsage / value comparison by churn status:')
print(usage_comp)

# Complaint proxy from churn categories and reasons
churn_only = df[df['is_churn'] == 1]
category_top = churn_only['Churn_Category'].value_counts().head(10)
reason_top = churn_only['Churn_Reason'].value_counts().head(10)

print('\nTop churn categories:')
print(category_top)
print('\nTop churn reasons:')
print(reason_top)

## 6. Churn Pattern Segmentation

In [ ]:
# Tenure bands
bins = [0, 6, 12, 24, 36, 60, np.inf]
labels = ['0-6', '7-12', '13-24', '25-36', '37-60', '61+']
df['tenure_band'] = pd.cut(df['Tenure_in_Months'], bins=bins, labels=labels, include_lowest=True)

tenure_summary = (
    df.groupby('tenure_band', observed=False)['is_churn']
      .agg(['count', 'mean'])
      .rename(columns={'count': 'customers', 'mean': 'churn_rate'})
)
tenure_summary['churn_rate'] = (tenure_summary['churn_rate'] * 100).round(2)
print('Churn by tenure band (%):')
print(tenure_summary)

# Contract and payment risk
for col in ['Contract', 'Payment_Method', 'Internet_Type']:
    t = df.groupby(col)['is_churn'].mean().sort_values(ascending=False)
    print(f'\nChurn rate by {col} (%):')
    print((t * 100).round(2))


## 7. Plots for Dashboard Inputs

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Tenure band churn
(tenure_summary['churn_rate']).plot(kind='bar', ax=axes[0], color='#1f77b4')
axes[0].set_title('Churn Rate by Tenure Band (%)')
axes[0].set_xlabel('Tenure Band')
axes[0].set_ylabel('Churn Rate (%)')

# Contract churn
contract_rate = (df.groupby('Contract')['is_churn'].mean() * 100).sort_values(ascending=False)
contract_rate.plot(kind='bar', ax=axes[1], color='#ff7f0e')
axes[1].set_title('Churn Rate by Contract (%)')
axes[1].set_xlabel('Contract')
axes[1].set_ylabel('Churn Rate (%)')

# Payment method churn
payment_rate = (df.groupby('Payment_Method')['is_churn'].mean() * 100).sort_values(ascending=False)
payment_rate.plot(kind='bar', ax=axes[2], color='#2ca02c')
axes[2].set_title('Churn Rate by Payment Method (%)')
axes[2].set_xlabel('Payment Method')
axes[2].set_ylabel('Churn Rate (%)')

plt.tight_layout()
plt.show()

## 8. Additional Visualizations
These visuals are useful for report screenshots and Power BI design alignment.

In [ ]:
# 1) Churn vs non-churn distribution
status_counts = df['Churn_Label'].value_counts()
color_map = {'Yes': '#d62728', 'No': '#2ca02c'}
bar_colors = [color_map.get(lbl, '#1f77b4') for lbl in status_counts.index]

fig, ax = plt.subplots(figsize=(6, 5))
status_counts.plot(kind='bar', color=bar_colors, ax=ax)
ax.set_title('Customer Distribution by Churn Status')
ax.set_xlabel('Churn Label')
ax.set_ylabel('Customer Count')
for i, v in enumerate(status_counts.values):
    ax.text(i, v + max(status_counts.values) * 0.01, str(int(v)), ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
# 2) Monthly charge distribution by churn label
fig, ax = plt.subplots(figsize=(8, 5))
for label, color in [('Yes', '#d62728'), ('No', '#2ca02c')]:
    s = df.loc[df['Churn_Label'] == label, 'Monthly_Charge'].dropna()
    if not s.empty:
        ax.hist(s, bins=30, alpha=0.45, label=f'Churn {label}', color=color)
ax.set_title('Monthly Charge Distribution by Churn Status')
ax.set_xlabel('Monthly Charge')
ax.set_ylabel('Frequency')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 3) Top churn reasons (churned customers only)
top_reasons = (
    df.loc[df['is_churn'] == 1, 'Churn_Reason']
      .value_counts()
      .head(10)
)

fig, ax = plt.subplots(figsize=(10, 6))
top_reasons.sort_values().plot(kind='barh', color='#9467bd', ax=ax)
ax.set_title('Top 10 Churn Reasons')
ax.set_xlabel('Churned Customer Count')
ax.set_ylabel('Churn Reason')
plt.tight_layout()
plt.show()

In [ ]:
# 4) Satisfaction score impact on churn rate
sat_rate = (df.groupby('Satisfaction_Score')['is_churn'].mean() * 100).sort_index()

fig, ax = plt.subplots(figsize=(7, 5))
sat_rate.plot(marker='o', linewidth=2, color='#1f77b4', ax=ax)
ax.set_title('Churn Rate by Satisfaction Score')
ax.set_xlabel('Satisfaction Score')
ax.set_ylabel('Churn Rate (%)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Key Insights and Business Actions

Use these outputs in your report and dashboard narrative:
1. Overall churn baseline and target segment sizes.
2. High churn concentration in short tenure and month-to-month contracts.
3. Higher churn among specific payment/internet groups.
4. Complaint and competitor-driven reasons dominate churn categories.

Recommended actions:
- Early-life retention program (0-12 months).
- Contract upgrade incentives (month-to-month to annual).
- Competitor defense bundles for vulnerable internet segments.
- Support quality initiatives for complaint-heavy churn reasons.